# 【総合演習】Antennaからの画像データのダウンロード  

### Selenium + BeautifulSoup + Requestsを組合せたスクレイピング

ここでは400もの提携メディアから流行のコンテンツを提供するキュレーションサービスであるAntenna( http://antenna.jp/ )を使って演習をします。

今後の商品開発に活かす為、どのようなノートPCがトレンドなのかを把握できるよう、検索ワード「ノートPC」で記事を検索し、検索結果のタイトルと詳細ページへのURL、画像ファイルを取得していきます。

### タスク1： Seleniumを用いてブラウザを起動し、Antennaのサイトを表示してください。その際に結果が確認しやすいよう、driver.maximize_window()を用いてブラウザを最大化して表示してください。

In [1]:
from selenium import webdriver
from time import sleep
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup

In [2]:
driver = webdriver.Chrome("C:\Test_Folder\chromedriver_win32\chromedriver")
target_url = "http://antenna.jp/"
driver.get(target_url)
driver.maximize_window()
sleep(3)

### タスク2： Antennaのサイトの検索欄に「ノートPC」と入力し、検索結果を表示してください。

In [3]:
search_text = driver.find_element_by_xpath('//input[@id="search-input"]')
# search_text = driver.find_element_by_css_selector('input#search-input')

search_text.send_keys('ノートPC')

search_button = driver.find_element_by_xpath('//input[@id="search-button"]')
# search_button = driver.find_element_by_css_selector('input#search-button')

search_button.click()
#search_button.submit()
#search_button.send_keys(Keys.ENTER)
sleep(3)

### タスク3： 検索結果が全て表示されるよう、画面スクロールしてください。

In [4]:
scroll_count = 20
for i in range(scroll_count):
    driver.find_element_by_tag_name('body').send_keys(Keys.END)
#    driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
    sleep(1)

### タスク4： 検索結果として表示された記事のタイトル、詳細ページへのURL、画像のURLを取得し、表示してください。<BR>また合わせて、最初の記事をNo.0として、記事に連番を振って表示してください。<BR>そして、画像のURLは、後からダウンロードできるよう、全てリストに格納してください。

In [5]:
i = 0
all_images = []

soup = BeautifulSoup(driver.page_source, "html.parser")
for article in soup.find_all("div", class_="article-view feed-article-view album-article"):   
#for article in soup.select("div.article-view.feed-article-view.album-article"):
    title = article.find("div", class_="title").text
    url = article.find("a", class_="thumbnail-content").attrs["href"]
    img =  article.find("img").attrs["src"]
    print("No." + str(i) + " : " + title)
    print(url)
    print(img, end="\n\n")    
    all_images.append(img)
    i += 1

No.0 : Windows 11がAndroidアプリに対応。この新機能が持つ意味とは？
https://antenna.jp/articles/13575141
https://staticx.antenna.jp/article_images/13575141_wide_5a18e4cc-2b03-4aca-ab2c-cdcbd1a6f734_.png

No.1 : ノートPCやタブレットの保管は縦置きがスタイリッシュです
https://antenna.jp/articles/13574497
https://staticx.antenna.jp/article_images/13574497_wide_5670e921-f71a-4060-ba8c-3dd5136b260a_.jpeg

No.2 : 売れてる「ライフハックツール」TOP10｜1位はAnkerのアレ！
https://antenna.jp/articles/13572331
https://staticx.antenna.jp/article_images/13572331_wide_2fe1ca02-0436-4c18-8d74-0393e41b0f39_.jpeg

No.3 : デイパックのPCスリーブに収納可能！ ニーモが放つ、折り畳み式テーブルが画期的。
https://antenna.jp/articles/13571696
https://staticx.antenna.jp/article_images/13571696_wide_84a59101-26f5-4c9e-8a05-736cc67eb007_.jpeg

No.4 : キズや水に強いイタリアンレザーを使用！軽さと耐久性を備えたトートバッグ
https://antenna.jp/articles/13570222
https://staticx.antenna.jp/article_images/13570222_wide_7d186da5-d42c-4759-97f1-84fcfd5d173b_.jpeg

No.5 : レトロデザインで夏の装いを引き立てる、「ヴェルサーチェ」のトートバッグ
https://antenna.jp/articles/13569856
https://staticx.antenn

### タスク5： リストに格納した画像のURLを元に、全ての画像をダウンロードしてください。画像のファイル名は、記事の連番としてください。

In [6]:
import requests
import os
import shutil

path = r"C:\Test_Folder\Antenna\photo"
for index, image_link in enumerate(all_images):
    filename = "image_" + str(index) + ".jpg"
    image_path = os.path.join(path, filename)
    response = requests.get(image_link, stream=True)
    with open(image_path, "wb") as file:
        shutil.copyfileobj(response.raw, file)